Testing the outputs in S3 bucket for the Project Data Lake


Preliminary imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import types 
from datetime import datetime
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, date_format, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, TimestampType, DateType
import os
import configparser

Read Config

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','AWS_SECRET_ACCESS_KEY')


Create Spark Session

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.5") \
        .getOrCreate()

Query Songs Parquet File from S3 bucket

In [4]:
#SOZCTXZ12AB0182364
output_data = "s3a://ri-datalake-project-s3/"
song_df = spark.read.parquet(os.path.join(output_data,'songs/'))

Using Spark SQL to show Songs table read from parquet files

In [5]:
song_df.createOrReplaceTempView("SongTable")
songSQL = spark.sql("select * from SongTable")
songSQL.show()

+------------------+--------------------+---------+----+------------------+
|           song_id|               title| duration|year|         artist_id|
+------------------+--------------------+---------+----+------------------+
|SOKTJDS12AF72A25E5|Drown In My Own T...|  192.522|   0|ARA23XO1187B9AF18F|
|SOEKAZG12AB018837E|I'll Slap Your Fa...|129.85424|2001|ARSVTNL1187B992A91|
|SOAFBCP12A8C13CC7D|King Of Scurf (20...|301.40036|1972|ARTC1LV1187B9A4858|
|SORRNOC12AB017F52B|The Last Beat Of ...|337.81506|2004|ARSZ7L31187FB4E610|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|118.07302|2005|AR73AIO1187B9AD57B|
|SODZYPO12A8C13A91E|Burn My Body (Alb...|177.99791|   0|AR1C2IX1187B99BF74|
|SOBRKGM12A8C139EF6|Welcome to the Pl...|821.05424|1985|ARXQBR11187B98A2CC|
|SOERIDA12A6D4F8506|I Want You (Album...|192.28689|2006|ARBZIN01187FB362CC|
|SOAPERH12A58A787DC|The One And Only ...|230.42567|   0|ARZ5H0P1187B98A1DD|
|SOSMJFC12A8C13DE0C|Is That All There...|343.87546|   0|AR1KTV21187B9ACD72|
|SONRWUU12AF

Query artists parquet files from S3 bucket

In [6]:
output_data = "s3a://ri-datalake-project-s3/"
artist_df = spark.read.parquet(os.path.join(output_data,'artists/'))

using spark sql to show artists table read from parquet files

In [7]:
artist_df.createOrReplaceTempView("ArtistTable")
artistSQL = spark.sql("select * from ArtistTable")
artistSQL.show()

+------------------+--------------------+--------------------+--------+----------+
|         artist_id|                name|            location|latitude| longitude|
+------------------+--------------------+--------------------+--------+----------+
|ARTC1LV1187B9A4858|  The Bonzo Dog Band|Goldsmith's Colle...| 51.4536|  -0.01802|
|ARA23XO1187B9AF18F|     The Smithereens|Carteret, New Jersey|40.57885| -74.21956|
|AR73AIO1187B9AD57B|   Western Addiction|   San Francisco, CA|37.77916|-122.42005|
|ARSVTNL1187B992A91|       Jonathan King|     London, England|51.50632|  -0.12714|
|AR5LMPY1187FB573FE|   Chaka Khan_ Rufus|         Chicago, IL|41.88415| -87.63241|
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|    null|      null|
|ARZ5H0P1187B98A1DD|          Snoop Dogg|      Long Beach, CA|33.76672| -118.1924|
|AR1KTV21187B9ACD72|            Cristina|     California - LA|34.05349|-118.24532|
|ARXQBR11187B98A2CC|Frankie Goes To H...|  Liverpool, England|    null|      null|
|ARY

Query users parquet file from s3 bucket 

In [8]:
output_data = "s3a://ri-datalake-project-s3/"
user_df = spark.read.parquet(os.path.join(output_data,'users/'))

In [9]:
user_df.createOrReplaceTempView("UsersTable")
userSQL = spark.sql("select * from UsersTable")
userSQL.show()

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     88|  Mohammad|Rodriguez|     M| free|
|     75|    Joseph|Gutierrez|     M| free|
|     69|  Anabelle|  Simpson|     F| free|
|     29|Jacqueline|    Lynch|     F| free|
|     68|    Jordan|Rodriguez|     F| free|
|      2|   Jizelle| Benjamin|     F| free|
|     61|    Samuel| Gonzalez|     M| free|
|     14|  Theodore|   Harris|     M| free|
|     40|    Tucker| Garrison|     M| free|
|     52|  Theodore|    Smith|     M| free|
|     17|  Makinley|    Jones|     F| free|
|    100|     Adler|  Barrera|     M| free|
|     12|    Austin|  Rosales|     M| free|
|     53|   Celeste| Williams|     F| free|
|     56|    Cienna|  Freeman|     F| free|
|     88|  Mohammad|Rodriguez|     M| paid|
|     73|     Jacob|    Klein|     M| paid|
|     38|    Gianna|    Jones|     F| free|
|     36|   Matthew|    Jones|     M| free|
|     77| Magdalene|   Herman|  

query time parquet file

In [10]:
#SOZCTXZ12AB0182364
output_data = "s3a://ri-datalake-project-s3/"
time_df = spark.read.parquet(os.path.join(output_data,'time/'))

In [12]:
time_df.createOrReplaceTempView("TimeTable")
timeSQL = spark.sql("select distinct start_time, hour, day ,week, day, week, weekday, year, month from TimeTable")
timeSQL.show()

+----------+----+---+----+---+----+-------+----+-----+
|start_time|hour|day|week|day|week|weekday|year|month|
+----------+----+---+----+---+----+-------+----+-----+
|2018-11-16|   0| 16|  46| 16|  46|      6|2018|   11|
|2018-11-26|   0| 26|  48| 26|  48|      2|2018|   11|
|2018-11-23|   0| 23|  47| 23|  47|      6|2018|   11|
|2018-11-07|   0|  7|  45|  7|  45|      4|2018|   11|
|2018-11-01|   0|  1|  44|  1|  44|      5|2018|   11|
|2018-11-05|   0|  5|  45|  5|  45|      2|2018|   11|
|2018-11-02|   0|  2|  44|  2|  44|      6|2018|   11|
|2018-11-25|   0| 25|  47| 25|  47|      1|2018|   11|
|2018-11-29|   0| 29|  48| 29|  48|      5|2018|   11|
|2018-11-17|   0| 17|  46| 17|  46|      7|2018|   11|
|2018-11-20|   0| 20|  47| 20|  47|      3|2018|   11|
|2018-11-15|   0| 15|  46| 15|  46|      5|2018|   11|
|2018-11-18|   0| 18|  46| 18|  46|      1|2018|   11|
|2018-11-08|   0|  8|  45|  8|  45|      5|2018|   11|
|2018-11-14|   0| 14|  46| 14|  46|      4|2018|   11|
|2018-11-1

query songplay parquet file

In [4]:
output_data = "s3a://ri-datalake-project-s3/"
songplay_df = spark.read.parquet(os.path.join(output_data,'songplay/'))

In [5]:
songplay_df.createOrReplaceTempView("SongPlayTable")
songplaySQL = spark.sql("select * from SongPlayTable")
songplaySQL.show()

+----------+------+-----+-------+---------+---------+--------------------+--------------------+------------+----+-----+
|start_time|userId|level|song_id|artist_id|sessionId|            location|           userAgent|song_play_id|year|month|
+----------+------+-----+-------+---------+---------+--------------------+--------------------+------------+----+-----+
|2018-11-15|    97| paid|   null|     null|      605|Lansing-East Lans...|"Mozilla/5.0 (X11...| 94489280512|2018|   11|
|2018-11-30|    49| paid|   null|     null|     1096|San Francisco-Oak...|Mozilla/5.0 (Wind...| 94489280513|2018|   11|
|2018-11-15|    80| paid|   null|     null|      602|Portland-South Po...|"Mozilla/5.0 (Mac...| 94489280514|2018|   11|
|2018-11-28|    39| free|   null|     null|      451|San Francisco-Oak...|"Mozilla/5.0 (Mac...| 94489280515|2018|   11|
|2018-11-19|    24| paid|   null|     null|      672|Lake Havasu City-...|"Mozilla/5.0 (Win...| 94489280516|2018|   11|
|2018-11-27|    88| paid|   null|     nu